In [1]:
import torch
import torch.nn as nn 


In [6]:
# LeNet with sub-classing 


class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()


    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2, stride=1),
        nn.Tanh()
    )
    self.avg_conv =   nn.AvgPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=0, stride=1),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, padding=0, stride=1),
        nn.Tanh()
    )

    self.flatten = nn.Flatten()

    self.dense1 = nn.Sequential(
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh()
    )
    self.dense2 = nn.Sequential(
        nn.Linear(in_features=84, out_features=10),
        nn.Tanh()
    )    


  def forward(self, x):
    print(f'input shape: {x.size()}')
    output = self.conv1(x)
    print(f'conv1 shape: {output.size()}')    
    output = self.avg_conv(output)
    print(f'conv1 shape: {output.size()}')
    output = self.conv2(output)
    print(f'conv2 shape: {output.size()}')
    output = self.conv3(output)
    print(f'conv3 shape: {output.size()}')
    output = self.flatten(output)
    print(f'flatten shape: {output.size()}')
    output = self.dense1(output)
    print(f'dense1 shape: {output.size()}')
    output = self.dense2(output)
    print(f'dense2 shape: {output.size()}')

    return output 

In [7]:
models = LeNet()
models

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
  )
  (avg_conv): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense1): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Tanh()
  )
  (dense2): Sequential(
    (0): Linear(in_features=84, out_features=10, bias=True)
    (1): Tanh()
  )
)

In [8]:
x = torch.randn(size=(32, 1, 28, 28))
predictions = models(x)

input shape: torch.Size([32, 1, 28, 28])
conv1 shape: torch.Size([32, 6, 28, 28])
conv1 shape: torch.Size([32, 6, 14, 14])
conv2 shape: torch.Size([32, 16, 5, 5])
conv3 shape: torch.Size([32, 120, 1, 1])
flatten shape: torch.Size([32, 120])
dense1 shape: torch.Size([32, 84])
dense2 shape: torch.Size([32, 10])


In [9]:
## LeNet with Hybrid Method



class ConvLayer(nn.Module):
  def __init__(self, in_filters, out_filters,padding, pool=True):
    super(ConvLayer, self).__init__()

    self.pool = pool

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=in_filters, out_channels=out_filters, kernel_size=5, padding = padding),
        nn.Tanh()
    )
    if self.pool:
      self.conv_pool = nn.AvgPool2d(kernel_size=2, stride=2)

  def forward(self, x):
    output = self.conv(x)
    if self.pool == True:
      output = self.conv_pool(output)

    return output


class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()


    self.conv1 = ConvLayer(1, 6, padding='same')
    self.conv2 = ConvLayer(6, 16, padding='valid')
    self.conv3 = ConvLayer(16, 120, padding='valid', pool=False)
    self.flatten = nn.Flatten()

    self.dense1 = nn.Sequential(
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh()
    )
    self.dense2 = nn.Sequential(
        nn.Linear(in_features=84, out_features=10),
        nn.Tanh()
    )    
    
  def forward(self, x):
    print(f'input size: {x.size()}')
    output = self.conv1(x)
    print(f'input size: {output.size()}')
    output = self.conv2(output)
    print(f'output size: {output.size()}')
    output = self.conv3(output)
    print(f'output size: {output.size()}')
    output = self.flatten(output)
    print(f'output size: {output.size()}')
    output = self.dense1(output)
    print(f'output size: {output.size()}')
    output = self.dense2(output)
    print('last output shape: ', output.size())
    return output 
  

In [10]:
mmodel = Model()
mmodel

Model(
  (conv1): ConvLayer(
    (conv): Sequential(
      (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=same)
      (1): Tanh()
    )
    (conv_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): ConvLayer(
    (conv): Sequential(
      (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=valid)
      (1): Tanh()
    )
    (conv_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv3): ConvLayer(
    (conv): Sequential(
      (0): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1), padding=valid)
      (1): Tanh()
    )
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense1): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Tanh()
  )
  (dense2): Sequential(
    (0): Linear(in_features=84, out_features=10, bias=True)
    (1): Tanh()
  )
)

In [11]:
x = torch.randn(size=(32, 1, 28,28))
predict=  mmodel(x)

input size: torch.Size([32, 1, 28, 28])
input size: torch.Size([32, 6, 14, 14])
output size: torch.Size([32, 16, 5, 5])
output size: torch.Size([32, 120, 1, 1])
output size: torch.Size([32, 120])
output size: torch.Size([32, 84])
last output shape:  torch.Size([32, 10])
